# Object detection of road safety from image tiles to the final database

## Creating image patches from image tiles

In [2]:
import numpy as np
import tifffile
from patchify import patchify
import os
from PIL import Image
from osgeo import gdal

# Define the input and output directories
input_dir_tiles = 'aerial_tiles_input/used5'
output_dir_patches = 'aerial_patches_output3'

# Define the patch size
patch_size = (1280, 1280)

# Loop over the GeoTIFF images in the input directory
for filename in os.listdir(input_dir_tiles):
    if filename.endswith('.tif'):
        # Load the GeoTIFF image using tifffile
        image = tifffile.imread(os.path.join(input_dir_tiles, filename))

        # Pad the input image to make its size divisible by the patch size
        padded_size = ((image.shape[0] + patch_size[0] - 1) // patch_size[0]) * patch_size[0], \
                      ((image.shape[1] + patch_size[1] - 1) // patch_size[1]) * patch_size[1]
        padding = ((0, padded_size[0] - image.shape[0]), (0, padded_size[1] - image.shape[1]), (0, 0))
        padded_image = np.pad(image, padding, mode='constant')

        # Extract patches using patchify
        patches = patchify(padded_image, (1280, 1280, 3), 1280)

        # Get the geotransform of the input image
        ds = gdal.Open(os.path.join(input_dir_tiles, filename))
        geotransform = ds.GetGeoTransform()

        # Loop over the patches and save them as JPEG/PNG images using Pillow
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = np.squeeze(patches[i, j])
                # Get the upper-left corner of the patch in geographic coordinates
                x = geotransform[0] + j * 1280 * geotransform[1] + i * 1280 * geotransform[2]
                y = geotransform[3] + j * 1280 * geotransform[4] + i * 1280 * geotransform[5]
                # Get the lower-right corner of the patch in geographic coordinates
                a = x + 1280 * geotransform[1]
                b = y + 1280 * geotransform[5]
                # Define the path and filename for the output file
                filename = f"{x}_{y}_{a}_{b}.jpg"
                path = os.path.join(output_dir_patches, filename)
                # Convert the patch to a PIL Image object and save as JPEG/PNG
                img = Image.fromarray(patch, mode='RGB')
                img.save(path, format='JPEG')

# Prediction using YOLOv8 model

In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git

In [3]:
!yolo task=detect mode=predict model=weights/yolov8_1280_v8/best.pt conf=0.25 source=aerial_patches_output3 save=False save_conf=True save_txt=True

Ultralytics YOLOv8.0.106 🚀 Python-3.8.10 torch-1.12.1+cu116 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs

image 1/5500 /data/private/Internship/aerial_patches_output3/100000.0_437078.4_100102.4_436976.0.jpg: 1280x1280 1 speedbump, 15.4ms
image 2/5500 /data/private/Internship/aerial_patches_output3/100000.0_437180.8_100102.4_437078.39999999997.jpg: 1280x1280 1 crosswalk, 3 speed30s, 2 speedbumps, 15.4ms
image 3/5500 /data/private/Internship/aerial_patches_output3/100000.0_437283.2_100102.4_437180.8.jpg: 1280x1280 (no detections), 15.4ms
image 4/5500 /data/private/Internship/aerial_patches_output3/100000.0_437385.6_100102.4_437283.19999999995.jpg: 1280x1280 (no detections), 15.4ms
image 5/5500 /data/private/Internship/aerial_patches_output3/100000.0_437488.0_100102.4_437385.6.jpg: 1280x1280 1 speedbump, 15.4ms
image 6/5500 /data/private/Internship/aerial_patches_output3/100000.0_437590.4_100102.4_437488.0.jpg: 1280x1

## Creat the database of predicted objects

In [4]:
# Define a function for tranforming center point coordinates to Dutch coordinates system

def transform_center_coordinates(local_x, local_y, dutch_x_tl, dutch_y_tl, dutch_x_br, dutch_y_br):
    dutch_x_coord = dutch_x_tl + (local_x * (dutch_x_br - dutch_x_tl))
    dutch_y_coord = dutch_y_tl + (local_y * (dutch_y_br - dutch_y_tl))
    return dutch_x_coord, dutch_y_coord

In [5]:
# Define a function to transform bounding box coordinates

def transform_bbox_coordinates(dutch_x_coord, dutch_y_coord, dutch_x_tl, dutch_y_tl, dutch_x_br, dutch_y_br, local_height, local_width):
    x_min = dutch_x_coord - ((dutch_x_br - dutch_x_tl) * local_width/2)
    x_max = dutch_x_coord + ((dutch_x_br - dutch_x_tl) * local_width/2)
    y_min = dutch_y_coord - ((dutch_y_br - dutch_y_tl) * local_height/2)
    y_max = dutch_y_coord + ((dutch_y_br - dutch_y_tl) * local_height/2)
    return x_min, x_max, y_min, y_max

### Create point shape file for centerpoints of all objects

In [6]:
import os

#defien the directory

directory = 'runs/detect/predict20/labels'

# Create a list to store the transformed points
transformed_points = []

# Define the category names
category_names = ['crossbike', 'crosswalk', 'schoolzone', 'speed30', 'speed50', 'speedbump']

# Iterate through the text files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)

        # Extract Dutch coordinate values from the text file name
        file_name = os.path.splitext(filename)[0]
        coordinates = file_name.split('_')
        ul_x, ul_y, br_x, br_y = map(float, coordinates)

        # Open the text file
        with open(file_path, 'r') as file:
            # Process each line in the text file
            for line in file:
                # Extract object information
                category, local_x, local_y, local_width, local_height, probability = map(float, line.strip().split())

                # Transform the coordinates
                dutch_x_coord, dutch_y_coord = transform_center_coordinates(local_x, local_y, ul_x, ul_y, br_x, br_y)

                # Transform the coordinates of bounding box

                x_min, x_max, y_min, y_max = transform_bbox_coordinates(dutch_x_coord, dutch_y_coord, ul_x, ul_y, br_x, br_y, local_height, local_width)

                # Get the category name based on the category index
                category_name = category_names[int(category)]

                # Store the transformed point with both category number and name in the list
                transformed_points.append([int(category), category_name, probability, dutch_x_coord, dutch_y_coord, x_min, x_max, y_min, y_max])

# Create a new CSV file
csv_file_path = os.path.join(directory, 'transformed_points_Capelle.csv')
with open(csv_file_path, 'w') as csv_file:
    # Write the header
    csv_file.write("cat_id,cat_name,prob,x_center,y_center\n")

    # Write the transformed points to the CSV file
    for point in transformed_points:
        csv_file.write(f"{point[0]},{point[1]},{point[2]},{point[3]},{point[4]}\n")


### Create polygon shape file for the boundingboxes of all objects|

In [7]:
import geopandas as gpd
from shapely.geometry import Polygon

# Create an empty GeoDataFrame
crs = 'EPSG:28992'  # Update with the appropriate CRS
geometry = []
categories = []
category_names = []
probabilities = []
x_mins = []
x_maxs = []
y_mins = []
y_maxs = []
# Iterate through the transformed points
for point in transformed_points:
    category = point[0]
    category_name = point[1]
    probability = point[2]
    x_min = point[5]
    x_max = point[6]
    y_min = point[7]
    y_max = point[8]
    
    # Create a polygon geometry from the coordinates
    polygon = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)])
    
    # Append the polygon and category information
    geometry.append(polygon)
    categories.append(category)
    category_names.append(category_name)
    probabilities.append(probability)
    x_mins.append(x_min)
    x_maxs.append(x_max)
    y_mins.append(y_min)
    y_maxs.append(y_max)
    

# Create the GeoDataFrame
data = {'cat_id': categories, 
        'cat_name': category_names, 
        'prob': probabilities, 
        'x_min': x_mins, 
        'x_max': x_maxs,
        'y_min': y_mins,
        'y_max': y_maxs,
        'geometry': geometry}
gdf = gpd.GeoDataFrame(data, crs=crs)

# Save the GeoDataFrame as a shapefile
output_file = 'runs/detect/predict20/labels/Capelle.shp'
gdf.to_file(output_file)
